<a href="https://colab.research.google.com/github/shaaagri/iat481-nlp-proj/blob/main/LLama2_vanilla_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
In this preparation notebook, we will set up a suitable variant of Llama2 LLM to be able to do some raw queries. We then will try to integrate it into a langchain-based pipeline and test it by asking the model some simple questions (a "vanilla" chatbot, missing yet any customization). This will help us to get ready for our project's next iteration (in another notebok) in which we will take this pipeline one step further by implementing RAG (Retrieval Augmented Generation).

*Note*: This notebook is partially based on `Chatbot_LLama_2.ipynb` from Maryiam's tutorials and also heavily borrows from `Run LLama-2 on Google Colab` tutorial by Muhammad Moin ([link](https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/blob/main/Run%20Llama2%20Google%20Colab/Llama_2_updated.ipynb])).

# Workflow

1. Gathering the Dependencies
2. Downloading and Loading the Model
3. Testing the Model with Raw Prompting

# Gathering the Dependencies


`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage (note: in the latest versions of llama-cpp-python the GGML format was replaced with `GGUF`, meaning we will need to adjust the code provided in the examples).



## Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGUF library by TheBloke.

We can see the different variations that Llama-2 GGUF has [here](https://huggingface.co/models?search=llama%202%20gguf).

In this case, we will use the model called [Llama-2-13B-Chat-GGUF](https://huggingface.co/TheBloke/Llama-2-13B-Chat-GGUF).


## Installing the Packages

In [3]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --upgrade --force-reinstall --verbose
!pip install huggingface_hub
!pip install llama-cpp-python

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Using cached llama_cpp_python-0.2.61-cp310-cp310-manylinux_2_35_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 74.5 MB/s eta 0:00:00
  Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Removing file or directory /usr/local/lib/python3.10/dist-packages/__pycache__/typing_extensions.cpython-310.pyc
      Removing file or directory /usr/local/lib/python3.10/dist-packages/typing_extensions-4.10.0.dist-info/
      Removing file or directory /usr/local/lib/python3.10/dist-packages/typing_extensions.py
      Successfully uninstalled typing_extensions-4.10.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.2

## Importing the libraries


In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

RuntimeError: Failed to load shared library '/usr/local/lib/python3.10/dist-packages/llama_cpp/libllama.so': libcuda.so.1: cannot open shared object file: No such file or directory

# Downloading and Loading the Model

Now that we have the required packages and libraries installed and imported, we can proceed to downloading and saving a quantized version of Llama-2-13b locally.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
models_root = '/content/drive/MyDrive/IAT481/481 Team Projs/NLP Project/Models/'

In [ ]:
%cd $models_root

/content/drive/.shortcut-targets-by-id/13FaXzyfvSXh_PD6h92caiYAKqKbPsFTR/481 Team Projs/NLP Project/Models


In [ ]:
#model_name_or_path = "TheBloke/Llama-2-7B-chat-GGUF"
#model_basename = "llama-2-7b-chat.Q4_K_M.gguf"

# 13b model seems to work just slightly slower than 7b, but its scores are better across all categories
# Hence, it is our final choice
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q4_K_M.gguf"

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.Q4_K_M.gguf:   0%|          | 0.00/7.87G [00:00<?, ?B/s]

The model should be available from now locally on our google drive (turns out, it gets saved outside of it at a cache dir, but anyway). Let's try to load it into our Colab's T4 GPU and see the stats.

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=-1 # Change this value based on your model and your GPU VRAM pool, -1 tries to move all layers to GPU
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

Everything seems to be okay and we are lucky that, thanks to quantization, we are able to use this LLM right from our Colab notebook, for free.

# Testing the Model with Raw Prompting

First, let's construct the default prompt template that contains a simple system prompt:

In [ ]:
def eval_default_prompt_templ(prompt):
  prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

  USER: %s

  ASSISTANT:
  '''

  return prompt_template % prompt

Now, without further ado, let's query the model with something and see what it gives us:

In [ ]:
prompt='Where Simon Fraser University is located?'

In [ ]:
response=lcpp_llm(prompt=eval_default_prompt_templ(prompt), max_tokens=256, temperature=0.4, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     655.04 ms
llama_print_timings:      sample time =      55.44 ms /    79 runs   (    0.70 ms per token,  1424.86 tokens per second)
llama_print_timings: prompt eval time =     426.34 ms /    19 tokens (   22.44 ms per token,    44.57 tokens per second)
llama_print_timings:        eval time =    4107.61 ms /    78 runs   (   52.66 ms per token,    18.99 tokens per second)
llama_print_timings:       total time =    4872.09 ms /    97 tokens


{'id': 'cmpl-b3be3505-5678-49e8-bee6-37a36948bef2', 'object': 'text_completion', 'created': 1712730623, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q4_K_M.gguf', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.\n\n  USER: Where Simon Fraser University is located?\n\n  ASSISTANT:\n   Sure thing! Simon Fraser University (SFU) has three campuses in British Columbia, Canada. The main campus is located in Burnaby, which is about 20 minutes from downtown Vancouver by car or public transit. The other two campuses are located in Surrey and Vancouver.\n   Is there anything else you'd like to know?", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 49, 'completion_tokens': 78, 'total_tokens': 127}}
